<a href="https://colab.research.google.com/github/iamemc/PD_02/blob/EMC/PD_03_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining / Prospecção de Dados

## Diogo F. Soares and Sara C. Madeira, 2020/21

# Project 2 - Clustering

## Logistics 

**_Read Carefully_**

**Students should work in teams of 2 or 3 people**. 

Individual projects might be allowed (with valid justification), but will not have better grades for this reason. 

The quality of the project will dictate its grade, not the number of people working.

**The project's solution should be uploaded in Moodle before the end of `April, 18th (23:59)`.** 

Students should **upload a `.zip` file** containing all the files necessary for project evaluation. 
Groups should be registered in [Moodle](https://moodle.ciencias.ulisboa.pt/mod/groupselect/view.php?id=139096) and the zip file should be identified as `PDnn.zip` where `nn` is the number of your group.

**It is mandatory to produce a Jupyter notebook containing code and text/images/tables/etc describing the solution and the results. Projects not delivered in this format will not be graded. You can use `PD_202021_P2.ipynb`as template. In your `.zip` folder you should also include an HTML version of your notebook with all the outputs** (File > Download as > HTML).

**Decisions should be justified and results should be critically discussed.** 

_Project solutions containing only code and outputs without discussions will achieve a maximum grade 10 out of 20._

## Dataset and Tools

In this project you should use [Python 3](https://www.python.org), [Jupyter Notebook](http://jupyter.org) and **[Scikit-learn](http://scikit-learn.org/stable/)**.

The dataset to be analysed is **`medulloblastoma_genes.csv`**. It includes 76 samples of medulloblastoma (MB) with respective expression levels of 54.675 genes measured in children with ages between 3 and 16 years. Medulloblastoma is a malignant childhood brain tumour comprising four discrete subgroups. 

In this project you will consider the labels of the samples included in the `labels.csv` file where samples are labelled as MB-CL or Other. In this case, we have 51 samples of classic medulloblastoma (MB-CL) and 25 other types (namely: 6 desmoplastic nodular, 17 anaplastic and 2 medullomyoblastoma).

In `medulloblastoma_genes.csv` each line represents a sample and each column represents a gene.


**The goal is to cluster samples and (ideally) find "MB-CL" groups and "Other MB" groups.** 

## Team Identification

**GROUP NNN**

Students:

* **Eduardo Carvalho - nº55881**
* **Filipe Santos - nº55142**
* **Ivo Oliveira - nº50301**

## 1. Load and Preprocess Dataset

At the end of this step you should have:
* a 76 rows × 54675 columns matrix, **X**, containing the values of the 54675 features for each of the 76 samples.
* a vector, **y**, with the 76 type of medulloblastoma, which you can use later to evaluate clustering quality.

In [26]:
# Importing relevant libraries
from pandas import read_csv, DataFrame, Series
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics.cluster import contingency_matrix

In [27]:
X=read_csv('/content/medulloblastoma_genes.csv', index_col=0)
X.shape

(76, 54675)

In [19]:
y=read_csv('/content/labels.csv', index_col=0)
y.shape

(76, 1)

In [20]:
# X, y = df.iloc[:, 1:].values, df2.iloc[:, 1].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                      test_size=0.3, 
#                      random_state=0, 
#                      stratify=y)

## 2. Dimensionality Reduction

As you already noticed the number of features (genes) is extremely high when compared to the number of objects to cluster (samples). In this context, you should perform dimensionality reduction, that is, reduce the number of features, in two ways:

* [**Removing features with low variance**](http://scikit-learn.org/stable/modules/feature_selection.html)

* [**Using Principal Component Analysis**](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

At the end of this step you should have two new matrices with the same number of rows, each with a different number of columns (features): **X_variance** and **X_PCA**. 

**Don't change X you will need it!**

### Low variance

In [21]:
var_thresh = VarianceThreshold(threshold=0)
var_X = var_thresh.fit_transform(X)

In [22]:
df_var=DataFrame(var_thresh.variances_, columns=['Variance'])\
                .sort_values(by='Variance', ascending=False)
df_var.head(30).plot.bar()
plt.xlabel('Features')
plt.ylabel('Variance');
plt.legend('')
plt.tight_layout()
plt.gcf().set_size_inches(20, 8)
plt.xticks(rotation = 45)
plt.show()

NameError: ignored

> By looking at the plot, we can identify the first 5 values as a good cutoff, as these features should a considerably higher variance than the rest.

In [ ]:
df_var.head(10)

> The threshold for the variance, to keep the 5 highest values, seems to be 6. We can now create a new VarianceThreshold object with this information.

In [ ]:
new_vt = VarianceThreshold(threshold=6)
new_vt.fit_transform(X)
mask=new_vt.get_support()
X_variance=X.loc[:, mask]
X_variance.head(3)

> Variance is influenced by the scale of the values. We can try to standardize the values, by dividing them for each feature's mean, and see if the results are significantly different.

In [ ]:
X_means = X / X.mean()
var_thresh_2 = VarianceThreshold(threshold=0)
var_X_2 = var_thresh_2.fit_transform(X_means)
df_var_2=DataFrame(var_thresh_2.variances_, columns=['Variance'])\
                .sort_values(by='Variance', ascending=False)
                
df_var_2.head(30).plot.bar()
plt.xlabel('Features')
plt.ylabel('Variance');
plt.legend('')
plt.tight_layout()
plt.gcf().set_size_inches(20, 8)
plt.xticks(rotation = 45)
plt.show()

> The results appear to be the same. The most obvious cutoff point is 2, with 5 being the next. So, we can continue with the previous selection. 

### PCA

In [ ]:
# Standardizing the features
std_X = StandardScaler().fit_transform(X)
std_X[:2]

In [ ]:
pca = PCA()
df_pca = DataFrame(pca.fit_transform(std_X))
df_pca.head(3)

In [ ]:
#DataFrame(pca.explained_variance_ratio_).head(30).plot.bar()
#plt.legend('')
#plt.tight_layout()
#plt.gcf().set_size_inches(10, 4)
#plt.xlabel('Principal Components')
#plt.ylabel('Explained Variance');


> Looking at the top 30 components in terms of explained variance, we can see some considerable drops at 3 points: from the 2nd to the 3rd, from the 3rd to the 4th and from the 6th to the 7th. Afterwards, the drop in explained variance is less noticeable.
>
> We will choose to keep 3 principal components as this number allows for easier comprehension of the dataset while still keeping a high amount of information.

In [ ]:
pca = PCA(n_components=3)
pca.fit(std_X)
X_PCA = pca.transform(std_X)
X_PCA[:3]

## 3. Clustering Samples using Partitional Clustering

Use **`K`-means** to cluster the samples:

* Cluster the original data (54.675 features): **X**.
    * Use different values of `K`.
    * For each value of `K` present the clustering by specifying how many samples MB-CL and Other are in each cluster.     
    For instance, `{0: {'MB-CL': 51, 'Other': 0}, 1: {'MB-CL': 0, 'Other': 25}}` is the ideal clustering that we aimed at obtained with K-means when `K=2`, where the first cluster has 51 MB-CL samples and 0 Other samples and the second cluster has 0 MB-CL samples and 25 Other samples.
    You can choose how to output this information.  **Tip**: You can explore the usage of contigency matrices.
    * What is the best value of `K` ? Justify using the clustering results and the [Silhouette score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html).

* Cluster the data obtained after removing features with low variance: **X_variance**.
    * Study different values of `K` as above.

* Cluster the data obtained after applying PCA: **X_PCA**.
    * Study different values of `K` as above.

* Compare the results obtained in the three datasets above for the best `K`. 
* Discuss the results.

## K-means - X

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans = kmeans.fit(std_X) #using standardized data
kmeans.cluster_centers_

In [ ]:
kmeans.labels_

In [ ]:
cm_X=contingency_matrix(y['class'], kmeans.labels_)
sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])
plt.show()
#most_bought.fig.set_figheight(12.5)
#most_bought.fig.set_figwidth(25)

> In this case we have 30 MB-CL samples in cluster 0 and 21 in cluster 1. We also have 7 "Other" samples in cluster 0 and 18 in cluster 1. The samples do not appear to be correctly separated according to the true labels.

### Determining ideal number of clusters by Elbow Plot

In [ ]:
cost =[]
for i in range(1, 21):
    KM = KMeans(n_clusters = i, max_iter = 500)
    KM.fit(std_X)
      
    # calculates squared error for the clustered points
    cost.append(KM.inertia_)   

# plot the cost against K values
plt.plot(range(1, 21), cost, color ='g', linewidth ='3')
plt.xlabel("Value of K")
plt.ylabel("Squared Error (Cost)")
plt.tight_layout()
plt.gcf().set_size_inches(10, 5)
plt.xticks(rotation = 45)
plt.ticklabel_format(style='plain')
plt.show() 

> Using the method of Elbow Plot does not seem to be very helpful, as there is no discernible cutoff point that we can use

### Determining ideal number of clusters by Silhouette Score

In [ ]:
silhouette_coefficients = []
for i in range(2, 11):
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(std_X)
  score = silhouette_score(std_X, kmeans.labels_)
  silhouette_coefficients.append(score)

plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.tight_layout()
plt.gcf().set_size_inches(20, 8)
plt.xticks(rotation = 45)
plt.show()

> The best number of clusters seems to be 2 as it has the highest value for the silhouette coefficient. In this case, we won't have to run the algorithm again, as we already used k=2 in the previous run.

for i in range(2,11):
  fig, ax = plt.subplots(figsize=(8,8))
  plt.suptitle('n_cluster = ' + str(i))
  kmeans = KMeans(n_clusters=i, random_state=0)
  kmeans = kmeans.fit(std_X)
  cm_X=contingency_matrix(y['class'], kmeans.labels_)
  sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])

### Running k-means for different values of k

In [ ]:
for i in range(2,11):
  fig, ax = plt.subplots(figsize=(8,6))
  plt.suptitle('n_cluster = ' + str(i))
  kmeans = KMeans(n_clusters=i, random_state=0)
  kmeans = kmeans.fit(std_X)
  cm_X=contingency_matrix(y['class'], kmeans.labels_)
  sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
             yticklabels=['MB-CL','Other'])
  plt.show()

> Looking at the contingency matrices for k between 2 and 10, we can see that, for k=4 and k=5, there are clusters where the predominant label is 'MB-CL' (cluster 3 for k=4, cluster 2 for k=5).
>
> Between these two, k=5 seems to be the case where this distinction is more evident, as the remaining clusters together don't have near as many 'MB-CL' labels as cluster 2 (18 vs 33).
>
> There isn't a k value where a cluster is predominantly composed of the label 'Other'.

> Based on the silhouette score and the contingency matrices, we conclude that, for the dataset with all features and standardized values, the ideal number of clusters is 2 - the silhouette score is the highest and there is an acceptable separation between labels.

## K-means - X_variance

In [ ]:
# Standardizing the features
std_X_var = StandardScaler().fit_transform(X_variance)

# K-means
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans = kmeans.fit(std_X_var)
kmeans.cluster_centers_

In [ ]:
kmeans.labels_

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.suptitle('n_cluster = 4')
cm_X_var=contingency_matrix(y['class'], kmeans.labels_)
sn.heatmap(cm_X_var, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])
plt.show()

> In this case we have 15 MB-CL samples in cluster 0 and 36 in cluster 1. We also have 7 "Other" samples in cluster 0 and 18 in cluster 1. Again, the samples do not appear to be correctly separated according to the true labels as both clusters appear to have a ratio of 2:1 in terms of labels (with predominance for label 'MB-CL', in both cases).

### Determining ideal number of clusters by Elbow Plot

In [ ]:
cost =[]
for i in range(1, 21):
    KM = KMeans(n_clusters = i, max_iter = 500)
    KM.fit(std_X_var)
      
    # calculates squared error for the clustered points
    cost.append(KM.inertia_)     
  
# plot the cost against K values
fig, ax = plt.subplots(figsize=(16,6))
plt.plot(range(1, 21), cost, color ='g', linewidth ='3')
plt.xlabel("Value of K")
plt.ylabel("Squared Error (Cost)")
plt.show() 

> This time we can actually spot an elbow shape in the plot, with the ideal value for k being between 4 and 6 (where the plot starts to stabilize).

### Determining ideal number of clusters by Silhouette Score

In [ ]:
silhouette_coefficients = []
for i in range(2, 11):
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(std_X_var)
  score = silhouette_score(std_X_var, kmeans.labels_)
  silhouette_coefficients.append(score)

fig, ax = plt.subplots(figsize=(16,6))
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

> The highest values for the silhouette coefficient are for k=10, 9 and. K=6 also shows a high score and it also matches with the observations made in elbow plot, appearing to be the most likely candidate thus far.

### Running k-means for different values of k

In [ ]:
for i in range(2,11):
  fig, ax = plt.subplots(figsize=(8,6))
  plt.suptitle('n_cluster = ' + str(i))
  kmeans = KMeans(n_clusters=i, random_state=0)
  kmeans = kmeans.fit(std_X_var)
  cm_X=contingency_matrix(y['class'], kmeans.labels_)
  sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])
  plt.show()

> There isn't really a k value where the the separation between labels is clear but k=6, as we have seen before, has two clusters with mostly 'MB-CL' labels. K=5 also shows the same level of separation.
>
> Again, there is no cluster where the label 'Other' is clearly predominant.

> Based on the silhouette score and the contingency matrices, we conclude that, for the dataset with only the high variance features and standardized values, the ideal number of clusters is 6 - the silhouette score is very high, the elbow plot starts to stabilize from that point forward and there is an acceptable separation between labels.

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
plt.suptitle('n_cluster = ' + str(i))
# K-means
kmeans = KMeans(n_clusters=6, random_state=0)
kmeans = kmeans.fit(std_X_var)
# continency matrix
cm_X_var=contingency_matrix(y['class'], kmeans.labels_)

sn.heatmap(cm_X_var, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])
plt.show()

## K-means - X_PCA

In [ ]:
# Standardizing the features
std_X_PCA = StandardScaler().fit_transform(X_PCA)

# K-means
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans = kmeans.fit(std_X_PCA)
kmeans.cluster_centers_

In [ ]:
kmeans.labels_

In [ ]:
cm_X_PCA=contingency_matrix(y['class'], kmeans.labels_)
fig, ax = plt.subplots(figsize=(8,6))
sn.heatmap(cm_X_PCA, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])
plt.show()

> We have 13 MB-CL samples in cluster 0 and 38 in cluster 1. There is a higher level of separation here than on the cases we have seen before. We also have 11 "Other" samples in cluster 0 and 14 in cluster 1. The samples for 'Other' still do not appear to be correctly separated according to the true labels.

### Determining ideal number of clusters by Elbow Plot

In [ ]:
cost =[]
for i in range(1, 21):
    KM = KMeans(n_clusters = i, max_iter = 500)
    KM.fit(std_X_PCA)
      
    # calculates squared error for the clustered points
    cost.append(KM.inertia_)     
  
# plot the cost against K values
fig, ax = plt.subplots(figsize=(16,6))
plt.plot(range(1, 21), cost, color ='g', linewidth ='3')
plt.xlabel("Value of K")
plt.ylabel("Squared Error (Cost)")
plt.show() 

> Like in the previous case, there is a noticeable elbow shape situated between k=4 and k=8. The ideal number of clusters should be somewhere between that interval.

### Determining ideal number of clusters by Silhouette Score

In [ ]:
silhouette_coefficients = []
for i in range(2, 11):
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(std_X_PCA)
  score = silhouette_score(std_X_PCA, kmeans.labels_)
  silhouette_coefficients.append(score)

fig, ax = plt.subplots(figsize=(16,6))
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

> The silhouette coefficient is higher for k values between 8 and 10 (9 being the highest). However, there is a considerable increase in the score for k=4, a value that is inside the interval determined by the elbow plot.
>
> For these reasons, k=4 seems to be the primary candidate for the number of clusters.

### Running k-means for different values of k

In [ ]:
for i in range(2,11):
  fig, ax = plt.subplots(figsize=(8,6))
  plt.suptitle('n_cluster = ' + str(i))
  kmeans = KMeans(n_clusters=i, random_state=0)
  kmeans = kmeans.fit(std_X_PCA)
  cm_X=contingency_matrix(y['class'], kmeans.labels_)
  sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])
  plt.show()

> Looking at the contingency matrices for k between 2 and 10, we can see that, for k=4 there are 2 clusters where the predominant label is 'MB-CL' (cluster 1 and 2), whereas for for k=5 there are 3 (clusters 1, 2 and 3).
>
> This seems to corroborate the previous assertion that k=4 is the best value for k, for the dataset after PCA.
>
> Again, there isn't a k value for which a cluster is predominantly composed of the label 'Other'.

> Based on the silhouette score, contingency matrices and elbow plot, we conclude that, for the dataset after PCA and standardized values, the ideal number of clusters is 4 - its silhouette score displays a big increase, the elbow plot seems to stabilize around that region and there is an acceptable separation between labels.

In [ ]:
# K-means
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans = kmeans.fit(std_X_PCA)
# continency matrix
cm_X_PCA=contingency_matrix(y['class'], kmeans.labels_)

fig, ax = plt.subplots(figsize=(8,6))
sn.heatmap(cm_X_PCA, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])
plt.show()

## Comparison of Results and Discussion

## 4. Clustering Samples using Hierarchical Clustering

Use a **Hierarchical Clustering Algorithm (HCA)** to cluster the samples: 

* Cluster the data in **X_variance**.
    * Use **different linkage metrics**.
    * Use different values of `K`.
    * For each linkage metric and value of `K` present the clustering by specifying how many MB-CL and Other samples are in each cluster as you did before. 
    * What is the best linkage metric and the best value of `K`? Justify using the clustering results and the [Silhouette score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html).

* Cluster the data in **X_PCA**.
    * Study different linkage metrics and different values of `K` as above.

* Compare the results obtained in the two datasets above for the best linkage metric and the best `K`. 
* Discuss the results

# Different linkage metrics for k=2

In [ ]:
from sklearn.cluster import AgglomerativeClustering
#linkage{‘ward’, ‘complete’, ‘average’, ‘single’}
#‘ward’ minimizes the variance of the clusters being merged.
#‘average’ uses the average of the distances of each observation of the two sets.
#‘complete’ or ‘maximum’ linkage uses the maximum distances between all observations of the two sets.
#‘single’ uses the minimum of the distances between all observations of the two sets.

std_X_var = StandardScaler().fit_transform(X_variance)
std_X_PCA = StandardScaler().fit_transform(X_PCA)

#hca = AgglomerativeClustering(linkage ="average", n_clusters=2)
# Grow the cluster hierarchy until K = 2 (dendogram)
#hca = hca.fit(std_X_var)
# Cluster memmberships
#hca.labels_
#cm_X_var1=contingency_matrix(y['class'], hca.labels_)
#sn.heatmap(cm_X_var1, annot=True, cmap="rocket", cbar=False, 
#           yticklabels=['MB-CL','Other'])
#plt.show()

linkage= ["average","ward","complete","single"]
for i in linkage:
  fig, ax = plt.subplots(figsize=(8,6))
  plt.suptitle('\nlinkage_metric = ' + str(i))
  hca = AgglomerativeClustering(linkage =i, n_clusters=6)
  hca = hca.fit(std_X_var)
  cm_X=contingency_matrix(y['class'], hca.labels_)
  sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])
  plt.show()

# Silhoutte score for various linkages

In [ ]:
linkage= ["average","ward","complete","single"]
for j in linkage:
  fig, ax = plt.subplots(figsize=(16,6))
  plt.suptitle('\nlinkage_metric = ' + str(j))
  silhouette_coefficients = []
  for i in range(2, 11):
    hca = AgglomerativeClustering(linkage =j, n_clusters=i)
    hca = hca.fit(std_X_var)
    score = silhouette_score(std_X_var, hca.labels_)
    silhouette_coefficients.append(score)

  plt.style.use("fivethirtyeight")
  plt.plot(range(2, 11), silhouette_coefficients)
  plt.xticks(range(2, 11))
  plt.xlabel("Number of Clusters")
  plt.ylabel("Silhouette Coefficient")
  plt.show()

## 5. Evaluating Clustering Results

#best k for linkage "ward"

In [ ]:
for i in range(2,11):
  fig, ax = plt.subplots(figsize=(8,6))
  plt.suptitle('\nlinkage_metric = ' + str(i))
  hca = AgglomerativeClustering(linkage ="ward", n_clusters=i)
  hca = hca.fit(std_X_var)
  cm_X=contingency_matrix(y['class'], hca.labels_)
  sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
           yticklabels=['MB-CL','Other'])
  plt.show()

### 5.1. Without Using Ground Truth

In this task you should compare the best results obtained using `K`-means and HCA 
1. **Without using ground truth**
2. **Using ground truth (`Medulloblastoma Type`)**.

**Choose one adequate measure** from those available by Sciki-learn (https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation) to evaluate the different clusterings. 

Discuss the results.

Calinski Harabasz metric for evaluation without prior knowledge of Ground Truth was chosen due to it's ease of calculation


Like most internal clustering criteria, Calinski-Harabasz is a heuristic device. The proper way to use it is to compare clustering solutions obtained on the same data, - solutions which differ either by the number of clusters or by the clustering method used.

There is no "acceptable" cut-off value. You simply compare CH values by eye. The higher the value, the "better" is the solution. If on the line-plot of CH values there appears that one solution give a peak or at least an abrupt elbow, choose it. If, on the contrary, the line is smooth - horizontal or ascending or descending - then there is no reason to prefer one solution to others.

CH criterion is based on ANOVA ideology. Hence, it implies that the clustered objects lie in Euclidean space of scale (not ordinal or binary or nominal) variables. If the data clustered were not objects X variables but a matrix of dissimilarities between objects then the dissimilarity measure should be (squared) euclidean distance (or, at worse, am other metric distance approaching euclidean distance by properties).

CH criterion is most suitable in case when clusters are more or less spherical and compact in their middle (such as normally distributed, for instance)1
. Other conditions being equal, CH tends to prefer cluster solutions with clusters consisting of roughly the same number of objects.

The Calinski-Harabasz index also known as the Variance Ratio Criterion, is the ratio of the sum of between-clusters dispersion and of inter-cluster dispersion for all clusters, the higher the score , the better the performances.

In [ ]:
from sklearn.metrics.cluster import calinski_harabasz_score

ch_scores = []
for i in range(2, 11):
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(std_X)
  score = calinski_harabasz_score(std_X, kmeans.labels_)
  ch_scores.append(score)

fig, ax = plt.subplots(figsize=(16,6))
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), ch_scores)#, label='Model length')
plt.suptitle("Calinski Harabasz Score of K-means")

#legend = ax.legend(loc='upper right', fontsize='x-small')
#legend.get_frame().set_facecolor('C0')

plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Calinski-Harabasz Score")
plt.show()

In [ ]:
ch2_scores1 = []
for i in range(2, 11):
  hca1 = AgglomerativeClustering(n_clusters=i)
  hca1 = hca1.fit(std_X_var)
  score = silhouette_score(std_X_var, hca1.labels_)
  ch2_scores1.append(score)

fig, ax = plt.subplots(figsize=(16,6))
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), ch2_scores1)#, label='Model length')
plt.suptitle("Calinski Harabasz Score of HCA")

#legend = ax.legend(loc='upper left', fontsize='x-small')
#legend.get_frame().set_facecolor('C0')

plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Calinski-Harabasz Score")
plt.show()

### 5.2. Using Ground Truth

**Choose one adequate measure** from those available by Sciki-learn (https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation) to evaluate the different clusterings. 

Discuss the results.

In [ ]:
from sklearn.metrics.cluster import silhouette_score, silhouette_samples
import numpy as np
import matplotlib.cm as cm
def drawSillouette(X, labels, header="" ):
    y_lower = 10
    clusters=list(set(labels))
    n_clusters=len(clusters)
    fig = plt.figure(figsize=(10,8))
    ax1 = plt.gca()
    ax1.set_xlim([-0.5, 1])
    ax1.set_ylim([0, len(X) + (n_clusters) * 3+ y_lower])
    sil_avg = silhouette_score(X, labels)
    for i,c in enumerate(clusters):
        silhouette_values = silhouette_samples(X, labels)
        cs_values = silhouette_values[labels == c]
        cs_values.sort()
        size_ci = cs_values.shape[0]
        y_upper = y_lower + size_ci
        color = cm.nipy_spectral(i / n_clusters) 
        ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, cs_values, facecolor=color, edgecolor="k", alpha=0.7)
        ax1.text(-0.05, y_lower + 0.5 * size_ci, str(c))
        y_lower = y_upper + 3  
    ax1.set_title("Silhouette plot " + header+ str())
    ax1.set_xlabel("\nSilhouette coefficient\n\n\n"+"Average Silhouette Score: "+str(sil_avg))
    ax1.set_ylabel("Clusters")
    ax1.axvline(x=sil_avg, c="r", linestyle=":")
    ax1.set_yticks([]) 
    plt.show()

drawSillouette(std_X, kmeans.labels_, "of K-means")

In [ ]:
drawSillouette(std_X, hca.labels_, "of HCA")

The Silhouette Coefficient is defined for each sample and is composed of two scores(shown in below), and a higher Silhouette Coefficient score relates to a model with better defined clusters.

The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.

The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.

## 6. Clustering Samples using Density-based Clustering

Use DBSCAN (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) or OPTICS (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.OPTICS.html) to cluster the samples.

Compare the results with those of K-means and HCA.

First of all, DBSCAN produces noise. And it's unclear how to correctly compute these indexes when there is noise. If you treat every noise point as its own cluster, the points will give you, e.g., a bad Silhouette, even when this is exactly the desire behavior. If you pretend noise is a cluster, the result will even be worse.

In [ ]:
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.preprocessing import normalize, StandardScaler

optics_model = OPTICS(min_samples = 10, xi = 0.05, min_cluster_size = 0.05).fit(std_X)

In [ ]:
print(std_X.max(), std_X.min())

In [ ]:
colors = list(map(lambda x: 'b' if x == 1 else 'r', kmeans.labels_)) #hca.labels_      kmeans.labels_
plt.scatter(std_X[:,0], std_X[:,1], c=colors, marker="o", picker=True)
#plt.scatter(std_X[:,0], std_X[:,-8])
#plt.scatter(std_X[:,0], std_X[:,9], color='r')
plt.grid(True)
plt.show()  

In [ ]:
colors = list(map(lambda x: 'b' if x == 1 else 'r', hca.labels_)) #hca.labels_      kmeans.labels_
plt.scatter(std_X[:,0], std_X[:,1], c=colors, marker="o", picker=True)
#plt.scatter(std_X[:,0], std_X[:,-8])
#plt.scatter(std_X[:,0], std_X[:,9], color='r')
plt.grid(True)
plt.show() 

## 7. Choose a Different Clustering Algorithm to Group the Samples

Choose **a clustering algorithm** besides `K`-means, HCA and DBSCAN/OPTICS to cluster the samples. 

**Groups of 3 People** must choose two different algorithms.

Justify your choice and compare the results with those of `K`-means, HCA and DBSCAN/OPTICS.

In [ ]:
from sklearn import cluster
#spectral = cluster.SpectralClustering(n_clusters=2,
#                                          eigen_solver='arpack',
#                                          affinity="nearest_neighbors")
#spectral = spectral.fit(std_X_var)
#spectral
#
#cm_X=contingencymatrix(y['class'], spectral.labels)
#sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
#         yticklabels=['MB-CL','Other'])
#plt.show()

##affinity_propagation = cluster.AffinityPropagation(damping=.9,
##                                                       preference=-50)
##affinity_propagation = affinity_propagation.fit(std_X_var)
##affinity_propagation
##cm_X=contingency_matrix(y['class'], affinitypropagation.labels)
##sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
##         yticklabels=['MB-CL','Other'])
##plt.show()

meanshift= cluster.MeanShift(bandwidth=None, seeds=None, bin_seeding=False, min_bin_freq=1, cluster_all=True, n_jobs=1)
meanshift= meanshift.fit(std_X_var)
cm_X=contingency_matrix(y['class'], meanshift.labels_)
sn.heatmap(cm_X, annot=True, cmap="rocket", cbar=False, 
         yticklabels=['MB-CL','Other'])
plt.show()

## 8. Conclusions

Draw some conclusions about this project work. Can you highlight some insights about meduloblastoma types? 

In [ ]:
# Write code in cells like this ...

Write text in cells like this...